# Binding site detection using DoGSiteScorer

## Imports

In [1]:
import requests

from biopandas.pdb import PandasPdb
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Jaime's DoGSiteScorer functions

In [3]:
def dogsite_scorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """This is the official API, but they only allow PDB codes, not custom ones..."""
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    # We have to query location for updates on the calculation
    return r.json()['location']

In [4]:
def dogsite_scorer_guess_binding_site(protein):
    """
    Use proteins.plus' DoGSiteScorer to retrieve most probable binding site in protein.
    """
    if len(protein) == 4:  # pdb code
        job_location = dogsite_scorer_submit_with_pdbid(protein)
    elif protein.endswith('.pdb'):
        job_location = dogsite_scorer_submit_with_custom_pdb(protein)
    else:
        raise ValueError("`protein` must be a PDB ID or a path to a .pdb file!")
    
    # Check when the calculation has finished
    while True:
        result = requests.get(job_location)
        result.raise_for_status()  # if it fails, it will stop here
        if result.status_code == 202:  # still running
            time.sleep(5)
            continue
        break
    
    # the residues files contain the geometric center and radius as a comment in the PDB file
    # first file (residues[0]) is the best scored pocket
    pdb_residues = requests.get(result.json()['residues'][0]).text
    for line in pdb_residues.splitlines():
        line = line.strip()
        if line.startswith('HEADER') and 'Geometric pocket center at' in line:
            fields = line.split()
            center = [float(x) for x in fields[5:8]]
            radius = float(fields[-1])
            break
    return center, radius  # this is what we need for our Vina calculation

## Try out API

### How to talk to server?

In [5]:
job_location = dogsite_scorer_submit_with_pdbid('3w32', 'A')

In [6]:
job_location

'https://proteins.plus/api/dogsite_rest/Yn2WyiSZp5wPh6oF6TjnKxag'

In [7]:
result = requests.get(job_location)

In [8]:
result

<Response [200]>

In [9]:
result.json()

{'status_code': 200,
 'result_table': 'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_desc.txt',
 'residues': ['https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_0_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_1_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_2_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_3_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_4_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_5_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_1_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_1_0_res.pdb',
  'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_1_1_res.pdb',
  'h

In [10]:
result.json().keys()

dict_keys(['status_code', 'result_table', 'residues', 'pockets', 'descriptor_explanation', 'parameters'])

### Check out result table

In [11]:
result.json()['result_table']

'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_desc.txt'

In [12]:
result_table = requests.get(result.json()['result_table']).text

In [13]:
result_table

'name\tlig_cov\tpoc_cov\tlig_name\tvolume\tenclosure\tsurface\tdepth\tsurf/vol\tlid/hull\tellVol\tell c/a\tell b/a\tsiteAtms\taccept\tdonor\thydrophobic_interactions\thydrophobicity\tmetal\tCs\tNs\tOs\tSs\tXs\tnegAA\tposAA\tpolarAA\tapolarAA\tALA\tARG\tASN\tASP\tCYS\tGLN\tGLU\tGLY\tHIS\tILE\tLEU\tLYS\tMET\tPHE\tPRO\tSER\tTHR\tTRP\tTYR\tVAL\tsimpleScore\tdrugScore\nP_0\t0.00\t0.00\t""\t1422.66\t0.10\t1673.75\t19.26\t1.176493329397045\t-\t-\t0.13\t0.67\t288\t  86\t  40\t  71\t0.36\t   0\t 198\t  45\t  41\t   4\t   0\t0.10\t0.13\t0.24\t0.53\t   4\t   5\t   2\t   5\t   2\t   2\t   1\t   5\t   0\t   3\t  12\t   3\t   2\t   3\t   3\t   1\t   2\t   1\t   1\t   5\t0.63\t0.810023\nP_0_0\t0.00\t0.00\t""\t599.23\t0.06\t540.06\t17.51\t0.9012566126529045\t-\t-\t0.14\t0.22\t131\t  35\t  13\t  25\t0.34\t   0\t  95\t  16\t  17\t   3\t   0\t0.03\t0.10\t0.28\t0.59\t   1\t   2\t   1\t   1\t   2\t   1\t   0\t   2\t   0\t   2\t   7\t   1\t   2\t   2\t   1\t   0\t   2\t   0\t   0\t   2\t0.59\t0.620201\nP_0_

In [14]:
# Split string into list of lists (=table)
result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]

In [15]:
result_table_split

[['name',
  'lig_cov',
  'poc_cov',
  'lig_name',
  'volume',
  'enclosure',
  'surface',
  'depth',
  'surf/vol',
  'lid/hull',
  'ellVol',
  'ell c/a',
  'ell b/a',
  'siteAtms',
  'accept',
  'donor',
  'hydrophobic_interactions',
  'hydrophobicity',
  'metal',
  'Cs',
  'Ns',
  'Os',
  'Ss',
  'Xs',
  'negAA',
  'posAA',
  'polarAA',
  'apolarAA',
  'ALA',
  'ARG',
  'ASN',
  'ASP',
  'CYS',
  'GLN',
  'GLU',
  'GLY',
  'HIS',
  'ILE',
  'LEU',
  'LYS',
  'MET',
  'PHE',
  'PRO',
  'SER',
  'THR',
  'TRP',
  'TYR',
  'VAL',
  'simpleScore',
  'drugScore'],
 ['P_0',
  '0.00',
  '0.00',
  '""',
  '1422.66',
  '0.10',
  '1673.75',
  '19.26',
  '1.176493329397045',
  '-',
  '-',
  '0.13',
  '0.67',
  '288',
  '  86',
  '  40',
  '  71',
  '0.36',
  '   0',
  ' 198',
  '  45',
  '  41',
  '   4',
  '   0',
  '0.10',
  '0.13',
  '0.24',
  '0.53',
  '   4',
  '   5',
  '   2',
  '   5',
  '   2',
  '   2',
  '   1',
  '   5',
  '   0',
  '   3',
  '  12',
  '   3',
  '   2',
  '   3',
  '

In [16]:
# Remove spaces
result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]

In [17]:
result_table_split

[['name',
  'lig_cov',
  'poc_cov',
  'lig_name',
  'volume',
  'enclosure',
  'surface',
  'depth',
  'surf/vol',
  'lid/hull',
  'ellVol',
  'ellc/a',
  'ellb/a',
  'siteAtms',
  'accept',
  'donor',
  'hydrophobic_interactions',
  'hydrophobicity',
  'metal',
  'Cs',
  'Ns',
  'Os',
  'Ss',
  'Xs',
  'negAA',
  'posAA',
  'polarAA',
  'apolarAA',
  'ALA',
  'ARG',
  'ASN',
  'ASP',
  'CYS',
  'GLN',
  'GLU',
  'GLY',
  'HIS',
  'ILE',
  'LEU',
  'LYS',
  'MET',
  'PHE',
  'PRO',
  'SER',
  'THR',
  'TRP',
  'TYR',
  'VAL',
  'simpleScore',
  'drugScore'],
 ['P_0',
  '0.00',
  '0.00',
  '""',
  '1422.66',
  '0.10',
  '1673.75',
  '19.26',
  '1.176493329397045',
  '-',
  '-',
  '0.13',
  '0.67',
  '288',
  '86',
  '40',
  '71',
  '0.36',
  '0',
  '198',
  '45',
  '41',
  '4',
  '0',
  '0.10',
  '0.13',
  '0.24',
  '0.53',
  '4',
  '5',
  '2',
  '5',
  '2',
  '2',
  '1',
  '5',
  '0',
  '3',
  '12',
  '3',
  '2',
  '3',
  '3',
  '1',
  '2',
  '1',
  '1',
  '5',
  '0.63',
  '0.810023'],

In [18]:
# Extract column names, index names, table body
column_names = result_table_split[0]
index_names = [i[0] for i in result_table_split[1:]]
table = [i[1:] for i in result_table_split[1:]]

In [19]:
table

[['0.00',
  '0.00',
  '""',
  '1422.66',
  '0.10',
  '1673.75',
  '19.26',
  '1.176493329397045',
  '-',
  '-',
  '0.13',
  '0.67',
  '288',
  '86',
  '40',
  '71',
  '0.36',
  '0',
  '198',
  '45',
  '41',
  '4',
  '0',
  '0.10',
  '0.13',
  '0.24',
  '0.53',
  '4',
  '5',
  '2',
  '5',
  '2',
  '2',
  '1',
  '5',
  '0',
  '3',
  '12',
  '3',
  '2',
  '3',
  '3',
  '1',
  '2',
  '1',
  '1',
  '5',
  '0.63',
  '0.810023'],
 ['0.00',
  '0.00',
  '""',
  '599.23',
  '0.06',
  '540.06',
  '17.51',
  '0.9012566126529045',
  '-',
  '-',
  '0.14',
  '0.22',
  '131',
  '35',
  '13',
  '25',
  '0.34',
  '0',
  '95',
  '16',
  '17',
  '3',
  '0',
  '0.03',
  '0.10',
  '0.28',
  '0.59',
  '1',
  '2',
  '1',
  '1',
  '2',
  '1',
  '0',
  '2',
  '0',
  '2',
  '7',
  '1',
  '2',
  '2',
  '1',
  '0',
  '2',
  '0',
  '0',
  '2',
  '0.59',
  '0.620201'],
 ['0.00',
  '0.00',
  '""',
  '201.73',
  '0.08',
  '381.07',
  '11.36',
  '1.8890100629554356',
  '-',
  '-',
  '0.17',
  '0.25',
  '51',
  '17',
  

In [20]:
result_table_df = pd.DataFrame(
    table,
    columns=column_names[1:],
    index=index_names
)

In [21]:
result_table_df.index.name = 'name'

In [22]:
result_table_df

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.00,0.00,"""""",1422.66,0.10,1673.75,19.26,1.176493329397045,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_0_0,0.00,0.00,"""""",599.23,0.06,540.06,17.51,0.9012566126529045,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_0_1,0.00,0.00,"""""",201.73,0.08,381.07,11.36,1.8890100629554356,-,-,0.17,0.25,51,17,9,10,0.28,0,36,6,7,2,0,0.08,0.17,0.25,0.50,1,1,0,1,1,1,0,0,0,0,3,1,1,0,0,0,1,0,0,1,0.17,0.174816
P_0_2,0.00,0.00,"""""",185.60,0.17,282.00,9.35,1.519396551724138,-,-,0.45,0.55,48,17,8,12,0.32,0,31,8,8,1,0,0.17,0.25,0.08,0.50,0,2,0,1,0,0,1,1,0,0,2,1,1,1,0,0,0,0,0,2,0.13,0.195695
P_0_3,0.00,0.00,"""""",175.30,0.15,297.42,9.29,1.6966343411294922,-,-,0.23,0.37,48,16,8,14,0.37,0,32,8,8,0,0,0.14,0.14,0.36,0.36,1,1,1,2,0,0,0,3,0,0,1,1,0,1,1,1,0,0,0,1,0.13,0.168845
P_0_4,0.00,0.00,"""""",170.37,0.08,390.10,11.99,2.2897223689616717,-,-,0.16,0.20,47,14,7,17,0.45,0,34,7,6,0,0,0.00,0.18,0.18,0.64,2,2,0,0,0,1,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0.15,0.223742
P_0_5,0.00,0.00,"""""",90.43,0.24,177.50,6.24,1.9628441888753732,-,-,0.70,0.89,26,8,6,5,0.26,0,16,6,4,0,0,0.12,0.25,0.25,0.38,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0.00,0.165232
P_1,0.00,0.00,"""""",708.99,0.13,1030.19,14.32,1.4530388298847656,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_1_0,0.00,0.00,"""""",496.90,0.11,739.17,12.72,1.4875628899174884,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489


This table still contains all values as floats - this would need to be changes if we'd really want to work with the data.

### Check out file with pocket residues

In [23]:
result.json()['residues'][0]

'https://proteins.plus/results/dogsite/Yn2WyiSZp5wPh6oF6TjnKxag/3w32_P_0_res.pdb'

In [24]:
pdb_residues = requests.get(result.json()['residues'][0]).text

In [25]:
pdb_residues

'HEADER\tOutput of DoGSiteScorer by A. Volkamer\nHEADER\tPocket 0 with 288 binding site atoms written.\nHEADER\tReferences: \nHEADER\tA. Volkamer et al. Analyzing the topology of active sites: on the prediction of pockets and subpockets. J. Chem. Inf. Model. 2010,50(11), 2041-52\nHEADER\tA. Volkamer et al. Combining global and local measures for structure-based druggability predictions. J. Chem. Inf. Model. 2012,52,360-372\nHEADER\tGeometric pocket center at  13.65  29.95   8.29 with max radius 20.00\nATOM      2  CA  GLN A 701      -0.291  31.978  -3.835  0.00  0.00           C\nATOM      3  C   GLN A 701       0.946  31.062  -3.957  0.00  0.00           C\nATOM      4  O   GLN A 701       0.876  29.863  -3.659  0.00  0.00           O\nATOM      6  CG  GLN A 701      -2.441  30.619  -3.562  0.00  0.00           C\nATOM     12  C   ALA A 702       3.775  30.322  -3.161  0.00  0.00           C\nATOM     13  O   ALA A 702       3.538  30.924  -2.110  0.00  0.00           O\nATOM     15  

#### Get geometric pocket center and radius

In [26]:
for line in pdb_residues.splitlines():
    line = line.strip()
    
    if line.startswith('HEADER') and 'Geometric pocket center at' in line:
        fields = line.split()
        center = [float(x) for x in fields[5:8]]
        radius = float(fields[-1])
        break

In [27]:
fields

['HEADER',
 'Geometric',
 'pocket',
 'center',
 'at',
 '13.65',
 '29.95',
 '8.29',
 'with',
 'max',
 'radius',
 '20.00']

In [28]:
center

[13.65, 29.95, 8.29]

In [29]:
radius

20.0

#### Get pocket PDB file text

This is what we will need. We will probably also need to locally download the pdb file.

In [30]:
ppdb = PandasPdb()

In [31]:
pdb_df = ppdb._construct_df(pdb_residues.splitlines(True))['ATOM']

In [32]:
pdb_df

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,blank_3,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,2,,CA,,GLN,,A,701,,,-0.291,31.978,-3.835,0.0,0.0,,,C,NaN,6
1,ATOM,3,,C,,GLN,,A,701,,,0.946,31.062,-3.957,0.0,0.0,,,C,NaN,7
2,ATOM,4,,O,,GLN,,A,701,,,0.876,29.863,-3.659,0.0,0.0,,,O,NaN,8
3,ATOM,6,,CG,,GLN,,A,701,,,-2.441,30.619,-3.562,0.0,0.0,,,C,NaN,9
4,ATOM,12,,C,,ALA,,A,702,,,3.775,30.322,-3.161,0.0,0.0,,,C,NaN,10
5,ATOM,13,,O,,ALA,,A,702,,,3.538,30.924,-2.110,0.0,0.0,,,O,NaN,11
6,ATOM,15,,N,,LEU,,A,703,,,4.401,29.139,-3.199,0.0,0.0,,,N,NaN,12
7,ATOM,16,,CA,,LEU,,A,703,,,5.009,28.552,-2.003,0.0,0.0,,,C,NaN,13
8,ATOM,17,,C,,LEU,,A,703,,,6.142,29.430,-1.494,0.0,0.0,,,C,NaN,14
9,ATOM,18,,O,,LEU,,A,703,,,6.222,29.735,-0.302,0.0,0.0,,,O,NaN,15


In [33]:
pdb_df.residue_number

0       701
1       701
2       701
3       701
4       702
5       702
6       703
7       703
8       703
9       703
10      703
11      705
12      705
13      705
14      705
15      705
16      705
17      718
18      718
19      718
20      718
21      719
22      720
23      720
24      720
25      721
26      721
27      721
28      722
29      722
30      722
31      722
32      723
33      723
34      723
35      723
36      723
37      723
38      723
39      723
40      724
41      724
42      724
43      724
44      726
45      726
46      726
47      731
48      731
49      731
50      740
51      740
52      743
53      743
54      743
55      743
56      743
57      744
58      744
59      744
60      745
61      745
62      745
63      745
64      745
65      745
66      745
67      761
68      762
69      762
70      762
71      762
72      762
73      762
74      762
75      762
76      765
77      765
78      765
79      765
80      765
81      766
82      766
83  

In [34]:
pdb_df['residue_number']

0       701
1       701
2       701
3       701
4       702
5       702
6       703
7       703
8       703
9       703
10      703
11      705
12      705
13      705
14      705
15      705
16      705
17      718
18      718
19      718
20      718
21      719
22      720
23      720
24      720
25      721
26      721
27      721
28      722
29      722
30      722
31      722
32      723
33      723
34      723
35      723
36      723
37      723
38      723
39      723
40      724
41      724
42      724
43      724
44      726
45      726
46      726
47      731
48      731
49      731
50      740
51      740
52      743
53      743
54      743
55      743
56      743
57      744
58      744
59      744
60      745
61      745
62      745
63      745
64      745
65      745
66      745
67      761
68      762
69      762
70      762
71      762
72      762
73      762
74      762
75      762
76      765
77      765
78      765
79      765
80      765
81      766
82      766
83  

In [35]:
pdb_df[['residue_number', 'residue_name']]

,residue_number,residue_name
0,701,GLN
1,701,GLN
2,701,GLN
3,701,GLN
4,702,ALA
5,702,ALA
6,703,LEU
7,703,LEU
8,703,LEU
9,703,LEU


In [36]:
pocket_residue_ids = pdb_df.residue_number.unique()

In [37]:
pocket_residue_ids

array([ 701,  702,  703,  705,  718,  719,  720,  721,  722,  723,  724,
        726,  731,  740,  743,  744,  745,  761,  762,  765,  766,  769,
        770,  771,  772,  774,  775,  776,  777,  778,  788,  789,  790,
        791,  792,  793,  794,  796,  797,  832,  833,  834,  836,  837,
        841,  842,  844,  852,  854,  855,  856,  857,  858,  860,  861,
        877,  997, 1001, 1013, 1014, 1016, 1017])

In [38]:
len(pocket_residue_ids)

62

## Try out other things

In [39]:
pd.read_json(
    'data/test.json', 
    orient='split'
)

,col 1,col 2
row 1,a,b
row 2,c,d
